<a href="https://colab.research.google.com/github/jinhyung426/deeplearning.ai/blob/main/tf_chap3_NLP_(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow

## Part 3. Natural Language Processing in TensorFlow

## (3) LSTM Models

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)
train_data, test_data = dataset['train'], dataset['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s, l in train_data:
	training_sentences.append(str(s.numpy()))
	training_labels.append(l.numpy())

for s, l in test_data:
	testing_sentences.append(str(s.numpy()))
	testing_labels.append(l.numpy())

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = "post"
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, truncating=trunc_type)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteQKDPVQ/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteQKDPVQ/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteQKDPVQ/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [2]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_reviews(text):
  return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_reviews(padded[1]))
print(training_sentences[1])

12 31 93 867 7 1256 6585 7961 421 365 2 26 14 9 988 1089 7 4 6728 6 276 5760 2587 2 81 6118 8029 2 139 1892 7961 5 5402 246 25 1 1771 350 5 369 56 5397 102 4 2547 3 4001 25 14 7822 209 12 3531 6585 7961 99 1 32 18 4762 3 19 184 3223 18 5855 1045 3 4232 3337 64 1347 5 1190 3 4459 8 614 7 3129 2 26 22 84 7020 6 71 18 4924 1160 161 50 2265 3 12 3983 2 12 264 31 2545 261 6 1 66 2 26 131 393 1 5846 6 15 5 473 56 614 7 1470 6
[  12   31   93  867    7 1256 6585 7961  421  365    2   26   14    9
  988 1089    7    4 6728    6  276 5760 2587    2   81 6118 8029    2
  139 1892 7961    5 5402  246   25    1 1771  350    5  369   56 5397
  102    4 2547    3 4001   25   14 7822  209   12 3531 6585 7961   99
    1   32   18 4762    3   19  184 3223   18 5855 1045    3 4232 3337
   64 1347    5 1190    3 4459    8  614    7 3129    2   26   22   84
 7020    6   71   18 4924 1160  161   50 2265    3   12 3983    2   12
  264   31 2545  261    6    1   66    2   26  131  393    1 5846    6
   15   

In [3]:
model = tf.keras.Sequential([
		tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),   # output is 2D, so we can use LSTM or GRU or RNN
		tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)), 
		
    # OR
		# tf.keras.layers.Conv1D(128, 5, activation='relu') - 128 FILTERS, 5 = WINDOW SIZE
    # tf.keras.layers.GlobalMaxPooling1D()
    # tf.keras.layers.Flatten()
    # but use LSTM since subwords have sequential meanings

		tf.keras.layers.Dense(6, activation='relu'),
		tf.keras.layers.Dense(1, activation='sigmoid')
	])

In [4]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                9600      
_________________________________________________________________
dense (Dense)                (None, 6)                 390       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 169,997
Trainable params: 169,997
Non-trainable params: 0
_________________________________________________________________


In [6]:
num_epochs = 15
history = model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/15
782/782 [==============================] - 16s 20ms/step - loss: 0.0499 - accuracy: 0.9840 - val_loss: 1.1519 - val_accuracy: 0.7514
Epoch 2/15
782/782 [==============================] - 16s 20ms/step - loss: 0.0438 - accuracy: 0.9845 - val_loss: 1.2580 - val_accuracy: 0.7516
Epoch 3/15
782/782 [==============================] - 16s 20ms/step - loss: 0.0332 - accuracy: 0.9888 - val_loss: 1.4343 - val_accuracy: 0.7523
Epoch 4/15
782/782 [==============================] - 16s 20ms/step - loss: 0.0248 - accuracy: 0.9918 - val_loss: 1.5904 - val_accuracy: 0.7547
Epoch 5/15
782/782 [==============================] - 15s 20ms/step - loss: 0.0252 - accuracy: 0.9914 - val_loss: 1.5653 - val_accuracy: 0.7536
Epoch 6/15
782/782 [==============================] - 15s 19ms/step - loss: 0.0188 - accuracy: 0.9939 - val_loss: 1.6169 - val_accuracy: 0.7526
Epoch 7/15
782/782 [==============================] - 15s 19ms/step - loss: 0.0131 - accuracy: 0.9958 - val_loss: 1.8651 - val_accuracy: